# TD 2 : Échantillonnage et estimateurs

Réalisé par : **Thomas REMY**

In [1]:
import os
import random

import src.td2.utils as utils

In [2]:
DATA_DIR = "../data/td2"
UNIFORM_DISTRIB_DATA = utils.lecture_valeurs(os.path.join(DATA_DIR, "uni_distrib.csv"))

NB_EXP = 1000000

## Expérience A : échantillonnage

### Calcul théorique

**Espérance**
$$\begin{align}
	\mathbb{E}(\mathcal{U}) = \mu &= \frac{a + b}{2}\\
	&= \frac{-1 + 1}{2} = 0
\end{align}$$

**Variance**
$$\begin{align}
	\text{var}(\mathcal{U}) = \sigma^2 &= \frac{(b - a)^2}{12}\\
	&= \frac{(-1 - 1)^2}{12} = \frac{(-2)^2}{12} = \frac{1}{3}
\end{align}$$

### Tirage d'un échantillon

Écriture d'une fonction permettant le tirage d'un échantillon dans une population sans remise.

In [3]:
def echantillon(n: int, population: list) -> list:
   len_pop = len(population)
   sample = []
   nb_samples = 0
   nb_elt_read = 0
   for elt in population:
       if random.uniform(0, 1) < (n - nb_samples) / (len_pop - nb_elt_read):
           sample.append(elt)
           nb_samples += 1
   return sample

### Calcul de la moyenne d'un tableau

Écriture d'une fonction permettant le calcul de la moyenne arithmétique d'un tableau.

In [4]:
def moyenne(tab: list) -> float:
    return sum(tab) / len(tab)

### Estimateur de la moyenne

On extrait des échantillons de tailles :

- 1000
- 10000
- 100000

On calcule ensuite l'estimateur de la moyenne pour chacun de ces échantillons, que l'on va comparer avec la valeur théorique $\mu = 0$.

In [5]:
small_sample = echantillon(1000, UNIFORM_DISTRIB_DATA)
moyenne(small_sample)

0.042852403513149936

In [6]:
medium_sample = echantillon(10000, UNIFORM_DISTRIB_DATA)
moyenne(medium_sample)

0.0016278240360882105

In [7]:
large_sample = echantillon(100000, UNIFORM_DISTRIB_DATA)
moyenne(large_sample)

0.0008229068544729542

On remarque ainsi que plus l'échantillon est grand, plus sa moyenne se rapproche de l'espérance théorique de la loi Uniforme.

## Expérience B : comparaison des estimateurs de la variance

### Préliminaires

Nous utilisaons ici des valeurs tirées aléatoirement d'une distribution uniforme.

#### Tirage de $n$ valeurs

On écrit ici une fonction réalisant le tirage de $n$ valeurs d'une distribution Uniforme.

In [8]:
def tirage(n: int) -> list:
    data = []
    for i in range(n):
        data.append(utils.source_aleatoire())
    return data

#### Calcul de l'estimateur de la variance naturelle

On écrit ici une fonction permettant le calcul de l'estimateur de la variance naturelle.



In [9]:
def variance(data: list, mu: float) -> float:
    somme = 0
    cpt = 0
    for elt in data:
        somme += (elt - mu) ** 2
        cpt += 1
    return somme / cpt

On va maintenant calculer la variance de 1,000,000 de valeurs tirées d'une distribution uniforme et comparer cette variance avec la valeur théorique $\sigma^2 = \frac{1}{3}$.

In [10]:
variance(UNIFORM_DISTRIB_DATA, 0)

0.3331836525553587

### Comparaison des estimateurs

On écrit ici 2 autres fonctions calculant l'estimateur de la variance :

- Variance biaisée
- Variance dé-biaisée en multipliant par un facteur \frac{n}{n - 1}

In [11]:
def variance_biaisee(data: list) -> float:
    mean = moyenne(data)
    return variance(data, mean)

def variance_non_biaisee(data: list) -> float:
    return (variance_biaisee(data) * len(data)) / (len(data) - 1)

Nous comparons maintenant les 3 estimateurs de la variance.

In [12]:
def compare_estimateurs(n: int) -> None:
    print(f"Comparaison des estimateurs de la variance : {n} expériences")
    print("*******************************************")
    variances = []
    variances_biaisee = []
    variances_non_biaisee = []
    
    for i in range(NB_EXP):
        data = tirage(n)
        variances.append(variance(data, 0))
        variances_biaisee.append(variance_biaisee(data))
        variances_non_biaisee.append(variance_non_biaisee(data))
    
    print(f"Quelques variances naturelles : {variances[:5]}")
    print(f"Quelques variances biaisées : {variances_biaisee[:5]}")
    print(f"Quelques variances non biaisées : {variances_non_biaisee[:5]}")
    
    mean_variances = moyenne(variances)
    mean_variances_biaisees = moyenne(variances_biaisee)
    mean_variances_non_biaisees = moyenne(variances_non_biaisee)
    
    print(f"Moyenne variances naturelles : {mean_variances}, erreur à la variance : {round(100 - (mean_variances / (1 / 3)) * 100, 3)}%")
    print(f"Moyenne variances biaisées : {mean_variances_biaisees}, erreur à la variance : {round(100 - (mean_variances_biaisees / (1 / 3)) * 100, 3)}%")
    print(f"Moyenne variances non biaisées : {mean_variances_non_biaisees}, erreur à la variance : {round(100 - (mean_variances_non_biaisees / (1 / 3)) * 100, 3)}%")
    print("*******************************************")
    print("*******************************************")

Nous allons réaliser dans un premier temps notre comparaison en réalisant 10 expériences, puis 100, puis 1000.

In [13]:
compare_estimateurs(10)
compare_estimateurs(100)
compare_estimateurs(1000)

Comparaison des estimateurs de la variance : 10 expériences
*******************************************
Quelques variances naturelles : [0.20234564740989552, 0.35207002723088954, 0.2867461973237259, 0.3852575665149244, 0.2721443188901323]
Quelques variances biaisées : [0.1995979589511227, 0.3435186415651756, 0.28413345350224806, 0.3830170036130034, 0.27154024622441936]
Quelques variances non biaisées : [0.2217755099456919, 0.38168737951686177, 0.31570383722472006, 0.42557444845889264, 0.3017113846937993]
Moyenne variances naturelles : 0.3334110950965815, erreur à la variance : -0.023%
Moyenne variances biaisées : 0.3000531714469486, erreur à la variance : 9.984%
Moyenne variances non biaisées : 0.3333924127188318, erreur à la variance : -0.018%
*******************************************
*******************************************
Comparaison des estimateurs de la variance : 100 expériences
*******************************************
Quelques variances naturelles : [0.33962719322718415

On remarque ici que l'erreur à la variance biaisée est $\sim 10\%$ pour $n = 10$ tirages par expérience, de $\sim 1\%$ pour $n = 100$ et de $\sim 0.1\%$ pour $n = 1000$. Ces valeurs correspondent bien à $\frac{1}{n}$.

## Expérience C : variance d'une somme de v.a. indépendantes

### Calcul théorique

Nous cherchons ici à calculer la variance de :

$$Y_n = \frac{X_1 + X_2 + \cdots + X_n}{n}$$

### Expérience

On écrit ici une fonction calculant la moyenne des moyennes ainsi que la variance des moyennes d'un grand nombre de tirages de $n$ valeurs.

In [14]:
def exp(n: int) -> None:
    print(f"Variance d'une moyenne de v.a. indépendantes : {n} expériences")
    print("*******************************************")
    means = []
    for i in range(NB_EXP):
        t = tirage(n)
        means.append(moyenne(t))
        
    print(f"Moyenne des moyennes : {moyenne(means)}")
    print(f"Variance des moyennes : {variance_non_biaisee(means)}")
    print("*******************************************")
    print("*******************************************")

On calcule maintenant la variance des moyennes pour :

- 10
- 20
- 50
- 100
- 500
- 1000

In [15]:
nb_tirages = [10, 20, 50, 100, 500, 1000]
for nb_tirage in nb_tirages:
    exp(nb_tirage)

Variance d'une moyenne de v.a. indépendantes : 10 expériences
*******************************************
Moyenne des moyennes : 0.0002457422335380282
Variance des moyennes : 0.03331166058989602
*******************************************
*******************************************
Variance d'une moyenne de v.a. indépendantes : 20 expériences
*******************************************
Moyenne des moyennes : 3.7605801400814765e-05
Variance des moyennes : 0.01668484884715359
*******************************************
*******************************************
Variance d'une moyenne de v.a. indépendantes : 50 expériences
*******************************************
Moyenne des moyennes : 0.00010031043302948306
Variance des moyennes : 0.006667525377726783
*******************************************
*******************************************
Variance d'une moyenne de v.a. indépendantes : 100 expériences
*******************************************
Moyenne des moyennes : 1.6434983480588965